In [2]:
import numpy as np
import random
from mnist import MNIST
import matplotlib.pyplot as plt

In [10]:
mndata = MNIST('../HW1/Dataset')
trainX = np.array(mndata.load_training()[0])[:50000]
trainY = np.array(mndata.load_training()[1])[:50000]

testX = np.array(mndata.load_testing()[0])[:1000]
testY = np.array(mndata.load_testing()[1])[:1000]

In [13]:
valIndices = np.random.choice(len(trainX), 2000)
nonValIndices = [x for x in range(len(trainX)) if x not in valIndices]

valX = trainX[valIndices]
valY = trainY[valIndices]

trainX = trainX[nonValIndices]
trainY = trainY[nonValIndices]

testX = np.array(mndata.load_testing()[0])[:2000]
testY = np.array(mndata.load_testing()[1])[:2000]

def feat(data,i):
    return data[i].tolist()

def oneHot(clas, noOfClasses):
    feat = np.zeros(noOfClasses)
    feat[clas] = 1;
    return feat

trnX = np.array([feat(trainX,i) for i in range(trainX.shape[0])])/256.0
trnY = np.array([oneHot(trainY[i], 10) for i in range(trainX.shape[0])])

tstX = np.array([feat(testX,i) for i in range(testX.shape[0])])/256.0
tstY = np.array([oneHot(testY[i], 10) for i in range(testX.shape[0])])

valX = np.array([feat(valX,i) for i in range(valX.shape[0])])/256.0
valY = np.array([oneHot(valY[i], 10) for i in range(valX.shape[0])])

In [14]:
trnY[0:10]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [54]:
def sigmoid(x):
    return 1.0/(1+np.exp(-1*x))

def lecun(x):
    return 1.7159*np.tanh(2.0*x/3)

def gradSigmoid(x):
    return x*(1-x)

def gradLecun(x):
    t = 2.0*x/3
    return 1.7159*(1-t**2)

def softmax(x):
    x = np.exp(x)
    x = x/x.sum(axis=1)[:, None]
    return x
    
maxValAcc = 0
lr = 0.00001
trnAcc = []
valAcc = []
tstAcc = []

# randomly initialize our weights with mean 0
n_hid_1 = 100
fan_in = 1.0#/np.sqrt(784)
# print fan_in
W1 = fan_in*(2*np.random.random((784,n_hid_1)) - 1)
# print W1
bias = fan_in*(2*np.random.random((n_hid_1)) - 1)
fan_in_h = 1.0#/np.sqrt(n_hid_1)
W2 = fan_in_h*(2*np.random.random((n_hid_1,10)) - 1)
bias2 = fan_in_h*(2*np.random.random((10)) - 1)

for j in xrange(300):
    indices = np.random.choice(len(trnX), len(trnX))
    tempY = trnY[indices]
    tempX = trnX[indices]
    A1 = np.dot(tempX, W1) + bias
    l1 = sigmoid(A1)
    
    A2 = np.dot(l1, W2) + bias2
    l2 = softmax(A2)

    # Errors in output layer
    d2 = (l2 - tempY)
    dbias2 = np.sum(d2, axis = 0)
        
    # Delta of W2
    dW2 = np.dot(l1.T, d2)

    # Errors in 1st hidden layer
    d1 = np.dot(d2, W2.T)*grad(l1)
    dbias = np.sum(d1, axis = 0)    
    
    # Delta W2
    dW1 = np.dot(tempX.T, d1)
    
    W2 -= lr*dW2
    bias2 -= lr*dbias2
    W1 -= lr*dW1
    bias -= lr*dbias
    
    prediction = softmax(np.dot(lecun(np.dot(valX, W1)+bias ), W2)+bias2)
    correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(valY, axis = 1), np.argmax(prediction, axis = 1))]
    valAcc.append(np.sum(correct)*100.0/len(valX))
    
    prediction = softmax(np.dot(lecun(np.dot(trnX, W1)+bias ), W2)+bias2)
    correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(trnY, axis = 1), np.argmax(prediction, axis = 1))]
    trnAcc.append(np.sum(correct)*100.0/len(trnX))   
    
    prediction = softmax(np.dot(lecun(np.dot(tstX, W1)+bias ), W2)+bias2)
    correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(tstY, axis = 1), np.argmax(prediction, axis = 1))]
    tstAcc.append(np.sum(correct)*100.0/len(tstX))   
    
#     if(valAcc[-1] > maxValAcc):
    print tstAcc[-1]
#         maxValAcc = valAcc[-1]

9.7
12.5
14.15
16.55
18.9
21.2
23.4
25.7
28.35
30.85
32.6
34.1
35.25
37.05
38.45
39.65
40.7
41.75
42.75
43.8
44.7
45.7
46.5
47.6
48.3
49.1
49.55
50.4
51.0
51.4
52.05
52.6
53.0
53.7
53.9
54.4
54.85
55.5
55.95
56.4
56.5
56.95
57.1
57.55
57.75
58.1
58.45
58.8
59.15
59.1
59.75
60.1
60.35
60.85
61.15
61.35
61.65
61.85
62.05
62.15
62.45
62.65
62.65
62.75
63.05
63.1
63.25
63.6
63.95
63.85
64.1
64.2
64.4
64.55
64.8
64.95
65.05
65.05
64.95
65.1
65.1
65.2
65.25
65.35
65.5
65.55
65.65
65.75
65.75
65.8
66.05
66.0
66.1
66.3
66.4
66.65
66.65
66.75
66.65
66.85
66.8
67.1
67.15
67.1
67.2
67.25
67.4
67.55
67.45
67.65
67.7
67.85
67.9
68.05
68.1
68.2
68.3
68.3
68.45
68.55
68.65
68.7
68.65
68.75
68.7
68.95
68.95
69.05
69.05
69.15
69.25
69.3
69.4
69.45
69.55
69.65
69.75
69.8
69.75
69.95
69.95
69.85
70.0
70.1
70.15
70.25
70.35
70.35
70.35
70.45
70.5
70.45
70.45
70.45
70.6
70.5
70.5
70.5
70.65
70.65
70.7
70.7
70.7
70.8
70.75
70.7
70.7
70.8
70.95
70.8
70.8
70.9
70.95
70.9
70.95
70.95
71.1
71.0
71.05
71.15
71.0

In [48]:
plt.plot([x+1 for x in range(len(trnAcc))], trnAcc, label = 'Training Accuracy')
plt.plot([x+1 for x in range(len(valAcc))], valAcc, label = 'Validation Accuracy')
plt.plot([x+1 for x in range(len(tstAcc))], tstAcc, label = 'Testing Accuracy')
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.legend(loc='lower right', shadow=True)
plt.show()